<a href="https://colab.research.google.com/github/Arindam75/graph_networks/blob/main/03_node_classification_cora_simple_gnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Practice Code](https://github.com/PacktPublishing/Hands-On-Graph-Neural-Networks-Using-Python/blob/main)

Github page for the textbook Hands on GNNs Using Python by Maxime Labonne

In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.8 MB/s eta 0:00:00


In [ ]:
from torch_geometric.datasets import Planetoid
import numpy as np
import pandas as pd

The cora dataset is made of 2708 nodes, where each node represents a publication. The edges are reference made from one node to another. Each publication has 1433 binary features, representing the presence/absence of a word. We want to classify the nodes into 7 categories.

In [ ]:
dataset = Planetoid(root=".", name="Cora")
data = dataset[0]

Processing...
Done!


## Load Data
Basic information about the data

In [ ]:
print(f'dataset {dataset}: {data.num_nodes} nodes, {data.num_edges} edges')
print(f'node feature shape: {data.x.shape}')
print(f'Number of features {data.num_node_features}')
print(f'Number of classes {data.y.max().item() + 1}')
print(f'class labels: {data.y.unique()}')

dataset Cora(): 2708 nodes, 10556 edges
node feature shape: torch.Size([2708, 1433])
Number of features 1433
Number of classes 7
class labels: tensor([0, 1, 2, 3, 4, 5, 6])


In [ ]:
print(f'Graph:')
print('------')
print(f'Edges are directed: {data.is_directed()}')
print(f'Graph has isolated nodes: {data.has_isolated_nodes()}')
print(f'Graph has loops: {data.has_self_loops()}')

Graph:
------
Edges are directed: False
Graph has isolated nodes: False
Graph has loops: False


In [ ]:
data_df = pd.DataFrame(data.x)
data_df['label'] = data.y
data_df.head()

,0,1,2,3,4,5,6,7,8,9,...,1424,1425,1426,1427,1428,1429,1430,1431,1432,label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3


## Vanilla Neural Network

We build a simple dense neural network with one hidden layer and just use the node features. Very important to note that , the input data is very sparse with just 1's and 0's.

In [ ]:
import torch
from torch.nn import Linear
import torch.nn.functional as F

In [ ]:
def accuracy(y_pred, y_true):

    return torch.sum(y_pred == y_true) / len(y_true)

class MLP(torch.nn.Module):
  def __init__(self, input_dim, hidden_lyr, output_dim):
    super().__init__()
    torch.manual_seed(12345)
    self.lin1 = Linear(input_dim, hidden_lyr)
    self.lin2 = Linear(hidden_lyr, output_dim)

  def forward(self, x):
    x = self.lin1(x)
    x = x.relu()
    x = F.dropout(x, p=0.5, training=self.training)
    x = self.lin2(x)
    return F.log_softmax(x, dim=1)

  def fit(self, data, epochs):
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(self.parameters(),
                                 lr=0.01,
                                 weight_decay=5e-4)

    self.train()
    for epoch in range(epochs):
      optimizer.zero_grad()
      out = self.forward(data.x)
      loss = criterion(out[data.train_mask], data.y[data.train_mask])
      acc = accuracy(out[data.train_mask].argmax(dim=1), data.y[data.train_mask])
      loss.backward()
      optimizer.step()

      if epoch%20 == 0:
        val_loss = criterion(out[data.val_mask], data.y[data.val_mask])
        val_acc = accuracy(out[data.val_mask].argmax(dim=1), data.y[data.val_mask])

        print(f'Epoch {epoch:>3} | Train Loss: {loss:.3f} | Train Acc:'
                      f' {acc*100:>5.2f}% | Val Loss: {val_loss:.2f} | '
                      f'Val Acc: {val_acc*100:.2f}%')

  @torch.no_grad()
  def test(self, data):
    self.eval()
    out = self.forward(data.x)
    acc = accuracy(out.argmax(dim=1)[data.test_mask], data.y[data.test_mask])
    return acc

In [ ]:
model = MLP(data.num_node_features, 128, dataset.num_classes)
model.fit(data, epochs=100)

Epoch   0 | Train Loss: 1.950 | Train Acc: 14.29% | Val Loss: 1.94 | Val Acc: 13.00%
Epoch  20 | Train Loss: 0.005 | Train Acc: 100.00% | Val Loss: 1.92 | Val Acc: 52.40%
Epoch  40 | Train Loss: 0.001 | Train Acc: 100.00% | Val Loss: 2.16 | Val Acc: 51.40%
Epoch  60 | Train Loss: 0.005 | Train Acc: 100.00% | Val Loss: 1.77 | Val Acc: 50.60%
Epoch  80 | Train Loss: 0.006 | Train Acc: 100.00% | Val Loss: 1.67 | Val Acc: 53.60%


In [ ]:
acc = model.test(data)
print(f'test set accuracy {acc*100:.4f}')

test set accuracy 56.9000


## Graph Neural Network

We try the same classification but, this time we will use a GNN. The first step is to build the dajacency matrix with self loops.



In [ ]:
from torch_geometric.utils import to_dense_adj

adjacency = to_dense_adj(data.edge_index)[0]

#include self loops
adjacency += torch.eye(len(adjacency))
adjacency

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 1.,  ..., 0., 0., 0.],
        [0., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 1.],
        [0., 0., 0.,  ..., 0., 1., 1.]])

In [ ]:
class VanillaGNNLayer(torch.nn.Module):
    def __init__(self, dim_in, dim_out, ad_mat):
        super().__init__()
        self.linear = Linear(dim_in, dim_out, bias=False)
        self.ad_mat = ad_mat

    def forward(self, x):
        x = self.linear(x)
        x = torch.sparse.mm(self.ad_mat, x)
        return x

class VanillaGNN(torch.nn.Module):

  def __init__(self, input_dim, hidden_lyr, output_dim, adj_matrix):
    super().__init__()
    torch.manual_seed(12345)
    self.adj_matrix = adj_matrix
    self.gnn1 = VanillaGNNLayer(input_dim, hidden_lyr,adj_matrix)
    self.gnn2 = VanillaGNNLayer(hidden_lyr, output_dim,adj_matrix)


  def forward(self, x):

    h = self.gnn1(x)
    h = torch.relu(h)
    h = self.gnn2(h)

    return F.log_softmax(h, dim =1)

  def fit(self, data, epochs):

    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(self.parameters(),
                                 lr=0.01,
                                 weight_decay=5e-4)
    self.train()
    for epoch in range(epochs):
      optimizer.zero_grad()
      out = self.forward(data.x)
      loss = criterion(out[data.train_mask], data.y[data.train_mask])
      acc = accuracy(out[data.train_mask].argmax(dim=1), data.y[data.train_mask])
      loss.backward()
      optimizer.step()

      if epoch%20 == 0:
        val_loss = criterion(out[data.val_mask], data.y[data.val_mask])
        val_acc = accuracy(out[data.val_mask].argmax(dim=1), data.y[data.val_mask])

        print(f'Epoch {epoch:>3} | Train Loss: {loss:.3f} | Train Acc:'
                      f' {acc*100:>5.2f}% | Val Loss: {val_loss:.2f} | '
                      f'Val Acc: {val_acc*100:.2f}%')

  @torch.no_grad()
  def test(self, data):
    self.eval()
    out = self.forward(data.x)
    acc = accuracy(out.argmax(dim=1)[data.test_mask], data.y[data.test_mask])
    return acc

In [ ]:
vgnn = VanillaGNN(data.num_node_features, 128, dataset.num_classes, adjacency)
print(vgnn)

VanillaGNN(
  (gnn1): VanillaGNNLayer(
    (linear): Linear(in_features=1433, out_features=128, bias=False)
  )
  (gnn2): VanillaGNNLayer(
    (linear): Linear(in_features=128, out_features=7, bias=False)
  )
)


In [ ]:
# Train
vgnn.fit(data, epochs=100)

Epoch   0 | Train Loss: 2.105 | Train Acc: 22.86% | Val Loss: 2.05 | Val Acc: 15.20%
Epoch  20 | Train Loss: 0.007 | Train Acc: 100.00% | Val Loss: 3.67 | Val Acc: 71.40%
Epoch  40 | Train Loss: 0.001 | Train Acc: 100.00% | Val Loss: 3.81 | Val Acc: 72.40%
Epoch  60 | Train Loss: 0.001 | Train Acc: 100.00% | Val Loss: 3.79 | Val Acc: 73.40%
Epoch  80 | Train Loss: 0.001 | Train Acc: 100.00% | Val Loss: 3.61 | Val Acc: 74.00%


In [ ]:
acc = vgnn.test(data)
print(f'test set accuracy {acc*100:.4f}')

test set accuracy 72.8000
